In [1]:
# 导入同花顺包和登录同花顺

In [1]:
import pandas as pd
from iFinDPy import *
THS_iFinDLogin("gdzq6888","199212")

C:\Users\Xuwei\anaconda3\Lib\site-packages\iFinDPy.pth


0

In [11]:
# 定义方程
## 基金编码的格式如下："003567.OF"
## 日期的格式如下：
##      年中："XXXX-06-30",
##      年末："XXXX-12-31"
def fund_limit_ratio(fundcode, time):
    #获取基金持股数据
    position_data = pd.DataFrame(THS_DP('fund_asset_allocation_stock_holding',fundcode+';'+time,'fund_code:Y,fund_name:Y,report:Y,stock_code:Y,stock_name:Y,hold_num_stock:Y,hold_num_change_stock:Y,hold_num_ratio_float_stock:Y,hold_stock_amount:Y,net_value_ratio:Y,stock_mv_ratio:Y,investment_type:Y,management_company:Y').data)

    #获取持仓股票代码
    stock_code = str(list(position_data['stock_code'])).strip("[]")

    #根据股票代码获取市值和基金规模
    ## 股票市值
    market_value = THS_BD(stock_code,'ths_market_value_stock',time).data
    market_value = market_value.rename(columns={"thscode": "stock_code"})
    ## 基金规模
    fund_value = THS_BD(fundcode, 'ths_mergesize_fund',time).data
    fund_value = fund_value.rename(columns={"thscode": "fund_code"})

    #将市值和股票持仓数据合并
    merged_data = pd.merge(position_data,market_value,on='stock_code',how='left')
    merged_data = pd.merge(merged_data,fund_value,on='fund_code',how='left')

    # 添加举牌线限额和加权举牌线限额
    merged_data['举牌线限额'] = merged_data['ths_market_value_stock']*0.05
    merged_data['加权举牌线限额'] = merged_data['举牌线限额']*merged_data['stock_mv_ratio']/100

    # 与基金净值进行对比并计算比值
    sum_weighted_limit = sum(merged_data['加权举牌线限额'])
    limit_ratio = (merged_data['ths_mergesize_fund'].mean())/sum_weighted_limit

    print("基金"+ fundcode + "的加权举牌限额为" + str(sum_weighted_limit) + ", 其净值为" + str(merged_data['ths_mergesize_fund'].mean()))
    print("基金"+ fundcode + "的基金净值与加权举牌限额的比值为" + str(limit_ratio))
    return



# Example

In [12]:
# 华夏行业景气混合
fund_limit_ratio('003567.OF','2022-06-30')

基金003567.OF的加权举牌限额为1141979857.4767132, 其净值为11166356687.780022
基金003567.OF的基金净值与加权举牌限额的比值为9.7780679883907


In [13]:
# 中欧医疗健康混合
fund_limit_ratio('003095.OF','2022-06-30')

基金003095.OF的加权举牌限额为7020335741.04723, 其净值为71081438409.62009
基金003095.OF的基金净值与加权举牌限额的比值为10.125076781444188
